In [ ]:
import sys
sys.path.append('./libs')
import util_modelo_final, util_bd, util_fasta
import warnings
warnings.filterwarnings("ignore")
import os
import shutil
from sklearn.externals.joblib import Parallel, delayed
from sklearn.externals.joblib import load
from IPython.display import display

#limpieza
if os.path.isdir("./data"):
    shutil.rmtree("./data")
os.mkdir("./data")
if os.path.isdir("./CPAT"):
    shutil.rmtree("./CPAT")
if os.path.isdir("./Diamond"):
    shutil.rmtree("./Diamond")
if os.path.isdir("./features"):
    shutil.rmtree("./features")
    
def generar_data_especie(row_especie):
    id_especie = str(row_especie[0])
    # lncRNA
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 0 AND flg_seleccionado = 1 AND id_especie = " + id_especie
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, "./data/Especie" + id_especie + ".lncRNA.fasta")
    # PCT
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 1 AND flg_seleccionado = 1 AND id_especie = " + id_especie
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, "./data/Especie" + id_especie + ".PCT.fasta")
    # CDS
    query = "SELECT f.cod_secuencia, cds.coding FROM secuencias_CDS cds JOIN secuencias_features f ON cds.id_especie = f.id_especie AND cds.cod_secuencia = f.cod_secuencia WHERE f.flg_pct = 1 AND f.flg_seleccionado = 1 AND f.id_especie = " + id_especie
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, "./data/Especie" + id_especie + ".CDS.fasta")

def proceso_especie(row_especie):
    generar_data_especie(row_especie)

def obtener_especies():
    query = "SELECT m.id_especie, m.especie FROM especies_seleccionadas s JOIN maestra_especies m ON s.especie = m.especie ORDER BY id_especie"
    especies = util_bd.resultados_query(query)
    for especie in especies:
        if not os.path.isfile("./modelos_referenciales/modelo_Especie{}.plk".format(especie[0])):
            yield especie

print("Iniciando proceso...")

print("Generación de archivos fasta para las 30 especies")
%time Parallel(n_jobs=4, verbose=0)(delayed(proceso_especie)(row_especie) for row_especie in obtener_especies())

scores = [['accuracy','precision','recall']]
tuned_parameters = [{'svc__kernel': ['rbf'], 'svc__gamma': [1e-3], 'svc__C': [0.1,0.5,0.9,2]}]

query = "SELECT m.id_especie, m.especie FROM especies_seleccionadas s JOIN maestra_especies m ON s.especie = m.especie ORDER BY id_especie"
especies_rows = util_bd.resultados_query(query)
especies = ["Especie" + str(especie[0]) for especie in especies_rows]

%time util_modelo_final.crear_modelo_final(especies, tuned_parameters, scores, 4)

#limpieza - excepto la carpeta de modelos generados
if os.path.isdir("./data"):
    shutil.rmtree("./data")
if os.path.isdir("./CPAT"):
    shutil.rmtree("./CPAT")
if os.path.isdir("./Diamond"):
    shutil.rmtree("./Diamond")
if os.path.isdir("./features"):
    shutil.rmtree("./features")
    
print("Proceso terminado...")

display(load("./modelo_final/cv_results_final.bin"))
display(load("./modelo_final/params_final.bin"))
display(load("./modelo_final/resultado_final.bin"))

Iniciando proceso...
Generación de archivos fasta para las 30 especies
CPU times: user 178 ms, sys: 69.5 ms, total: 247 ms
Wall time: 4min 42s
Iniciando creación del modelo final
Prepocesamiento de transcritos previos al entrenamiento del modelo final
